In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'/home/ken/Work/kidney-classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [6]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        
        return data_ingestion_config

In [7]:
import zipfile
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size
import gdown

In [25]:
class DataIngestion:
    def __init__(self, config= DataIngestionConfig):
        self.config = config
        
    def download_file(self) ->str:
        """Fetch data from the URL"""
        try:
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")
            
            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            logger.info(f"Download url {prefix+file_id}")
            gdown.download(prefix+file_id, zip_download_dir)
            
            logger.info(f"Downloaded data from {dataset_url} into {zip_download_dir}")
        
        except Exception as e:
            raise e
        
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [26]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-22 13:38:10,555: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-22 13:38:10,556: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-22 13:38:10,557: INFO: common: created directory at: artifacts]
[2024-02-22 13:38:10,558: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-22 13:38:10,559: INFO: 4161349440: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into file artifacts/data_ingestion/data.zip]
[2024-02-22 13:38:10,559: INFO: 4161349440: file id 1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3]
[2024-02-22 13:38:10,560: INFO: 4161349440: Download url https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=740b33e9-0f48-451e-8a58-c792e14c14ce
To: /home/ken/Work/kidney-classification/artifacts/data_ingestion/data.zip
100%|██████████| 57.7M/57.7M [00:06<00:00, 9.53MB/s]

[2024-02-22 13:38:17,913: INFO: 4161349440: Downloaded data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into artifacts/data_ingestion/data.zip]
